In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime, time
from datetime import timedelta, date
from statistics import mean
from scipy import stats

In [ ]:
#     https://fyers123.in
start = datetime.date.today() - timedelta(days = 1)
end = datetime.date.today()
day = end.strftime("%d")
month = end.strftime("%m")
year = end.year
# print(end)
yesterday = datetime.date.today() - timedelta(days = 1)

if(end.strftime('%a') == 'Sat'):
    end = date.today() - timedelta(days = 1)
    day = end.strftime("%d")
    month = end.strftime("%m")
    year = end.year
    yesterday = datetime.date.today() - timedelta(days = 2)
elif(end.strftime('%a') == 'Sun'):
    end = datetime.date.today() - timedelta(days = 2)
    day = end.strftime("%d")
    month = end.strftime("%m")
    year = end.year
    yesterday = datetime.date.today() - timedelta(days = 3)

data_path = 'F:/stock market/Quant/Jupiter/Data/'
delivery_path = 'F:/stock market/Quant/Jupiter/Deliverables/data/'

end = f'{year}-{month}-{day}'
print(end)
df = pd.read_csv(f'{data_path}NIFTY 500/NIFTY500.csv')
# df = df['Symbol']
ticker_columns = ['Symbol',
                  'Price', 
                  '1M Change',
                  '1M Percentile',
                  '1W Change',
                  '1W Percentile', 
                  '1D Change', 
                  '1D Percentile',
                  '1M Volume Change',
                  '1M Volume Percentile', 
                  '1W Volume Change',
                  '1W Volume Percentile',
                  '1D Volume Change', 
                  '1D Volume Percentile',
                  'HQM Score']

spike_columns = ['Date',
                 'Symbol',
                 'Entry Price',
                 'Delivery Percentage Change',
                 'Delivery Percentage Percentile',
                 'Delivery Percentage Improvement Change',
                 'Delivery Percentage Improvement Percentile',
                 'Average Trade Worth Improvement Change',
                 'Average Trade Worth Improvement Percentile',
                 'Delivery Turnover Improvement Change',
                 'Delivery Turnover Improvement Percentile',
                 'Change Percentage',
                 'HQM Score']

spike_df = pd.DataFrame(columns = spike_columns)
ticker_df = pd.DataFrame(columns = ticker_columns)
# ticker_df.loc['1','Symbol'] = 'NIFTY'
for i in range(0, len(df)):
    symbol = df.loc[i,'Symbol']
    try:
#         single_df = yf.download(f'{symbol}.NS', start, end)
        single_df = pd.read_csv(f'{delivery_path}{symbol}.csv').set_index('Date')
#         single_df['Date'] = pd.to_datetime(single_df['Date'])
#         single_df.set_index('Date')
    except:
        continue
#     single_df['Date'] = pd.to_datetime(single_df['Date'])
#     single_df.reset_index(drop = True, inplace = True)
#     single_df.set_index('Date', drop = True)
    single_df['Volume Avg'] = single_df['Volume'].rolling(20).mean()
    single_df['Delivery Worth'] = single_df['Delivered Volume'] * single_df['Close']
    single_df['Trade Worth'] = single_df['Total Traded Value'] / single_df['Total Trades']
    single_df['Average Trade Worth'] = single_df['Trade Worth'].pct_change(periods = 1)*100
    single_df['Delivery Worth Change'] = single_df['Delivery Worth'].pct_change(periods = 1)*100
    single_df['1M Change'] = single_df['Close'].pct_change(periods = 21)*100
    single_df['1W Change'] = single_df['Close'].pct_change(periods = 5)*100
    single_df['1D Change'] = single_df['Close'].pct_change(periods = 1)*100
    single_df['1M Volume Avg'] = single_df['Delivery Percentage'].pct_change(periods = 21)*100
    single_df['1W Volume Avg'] = single_df['Delivery Percentage'].pct_change(periods = 5)*100
    single_df['1D Volume Avg'] = single_df['Delivery Percentage'].pct_change(periods = 1)*100
    single_df['1D NVolume Avg'] = single_df['Volume'].pct_change(periods = 1)*100
    
    try:
        price = single_df.loc[end, 'Close']
        One_MChange = single_df.loc[end, '1M Change']
        One_WChange = single_df.loc[end, '1W Change']
        One_DChange = single_df.loc[end, '1D Change']
        One_MVChange = single_df.loc[end, '1M Volume Avg']
        One_WVChange = single_df.loc[end, '1W Volume Avg']
        One_DVChange = single_df.loc[end, '1D Volume Avg']
    except:
        continue
    
    ticker_df = ticker_df.append(pd.Series([symbol,
                                           price,
                                           One_MChange,
                                           'N/A',
                                           One_WChange,
                                           'N/A',
                                           One_DChange,
                                           'N/A',
                                           One_MVChange,
                                           'N/A',
                                           One_WVChange,
                                           'N/A',
                                           One_DVChange,
                                           'N/A',
                                           'N/A'],
                                          index =ticker_columns),
                                ignore_index = True)
    Open = single_df.loc[end, 'Open']
    Close = single_df.loc[end, 'Close']
    Volume = single_df.loc[end, 'Volume']
    Avg_Volume = single_df.loc[end, 'Volume Avg']
    Delivered_Volume = single_df.loc[end, 'Delivered Volume']
    Delivery_Worth = single_df.loc[end, 'Delivery Worth']
    Delivery_Worth_Change = single_df.loc[end, 'Delivery Worth Change']
    Delivery_pct = single_df.loc[end, '1D Volume Avg']
    Delivery_Percentage = single_df.loc[end, 'Delivery Percentage']
    Average_Trade_Worth = single_df.loc[end, 'Average Trade Worth']
    

    if((Close > Open) and (Volume > Avg_Volume) and (Delivery_Worth > 5000000) and (Delivery_Percentage > 25.00) and (Delivery_Worth_Change > 25.00) and (Average_Trade_Worth > 50.00)):
        spike_df = spike_df.append(pd.Series([end,
                                              symbol,
                                              price,
                                              Delivery_Percentage,
                                              'N/A',
                                              Delivery_pct,
                                              'N/A',
                                              Average_Trade_Worth,
                                              'N/A',
                                              Delivery_Worth_Change,
                                              'N/A',
                                              'N/A',
                                              'N/A'],
                                            index = spike_columns),
                                  ignore_index = True)
#         print(symbol)
#         print(Close > Open)
#         print(Volume > Avg_Volume)
#         print(Delivery_Worth > 5000000)
#         print(Delivery_Percentage > 25.00)
#         print(Delivery_Worth_Change > 25.00)
#         print(Average_Trade_Worth > 50.00)

time_periods = ['1M',
               '1W',
               '1D',
               '1M Volume',
               '1W Volume',
               '1D Volume']

for row in ticker_df.index:
    for time_period in time_periods:
        ticker_df.loc[row, f'{time_period} Percentile'] = stats.percentileofscore(ticker_df[f'{time_period} Change'], ticker_df.loc[row, f'{time_period} Change'])/100

for row in ticker_df.index:
    momentum_percentile = []
    for time_period in time_periods:
        momentum_percentile.append(ticker_df.loc[row, f'{time_period} Percentile'])
    ticker_df.loc[row, 'HQM Score'] = mean(momentum_percentile)

ticker_df = ticker_df.sort_values(by = 'HQM Score', ascending = False)
ticker_df.reset_index(drop=True, inplace=True)

delivery_methods = ['Delivery Percentage',
                   'Delivery Percentage Improvement',
                   'Average Trade Worth Improvement',
                   'Delivery Turnover Improvement']

for row in spike_df.index:
    for time_period in delivery_methods:
        spike_df.loc[row, f'{time_period} Percentile'] = stats.percentileofscore(spike_df[f'{time_period} Change'], spike_df.loc[row, f'{time_period} Change'])/100

for row in spike_df.index:
    momentum_percentile = []
    for time_period in delivery_methods:
        momentum_percentile.append(spike_df.loc[row, f'{time_period} Percentile'])
    spike_df.loc[row, 'HQM Score'] = mean(momentum_percentile)

spike_df = spike_df.sort_values(by = 'HQM Score', ascending = False)
spike_df.reset_index(drop=True, inplace=True)

# ticker_df.round(2)
# single_df.round(2)
# single_df.columns
spike_df.round(2)
# spike_df.to_csv(f'{}')


2021-07-22
